In [2]:
# make sure to use the right version of pytorch when testing on nvidia/amd
import torch
import torch.nn.functional as F
torch.set_printoptions(sci_mode=False)

In [2]:
import math
M = 32


def flashattn(Q: torch.Tensor, K: torch.Tensor, V: torch.Tensor) -> torch.Tensor:
    assert Q.shape == K.shape == V.shape
    assert len(Q.shape) == 2

    K = K.T

    # seq length, inner dim of representations
    N, d = Q.shape

    # of kv vectors per tile
    bc = math.ceil(M/(4*d))
    tc = math.ceil(N/bc)
    K_shared = torch.empty((d, bc), dtype=Q.dtype)
    V_shared = torch.empty((bc, d), dtype=Q.dtype)

    # of q vectors per tile
    br = min(math.ceil(M/(4*d)), d)
    tr = math.ceil(N/br)
    Q_shared = torch.empty((br, d), dtype=Q.dtype)

    # print(f'bc={bc}, br={br}')
    # print(f'tc={tc}, tr={tr}')

    # output matrix
    O = torch.zeros_like(Q)
    O_shared = torch.empty_like(Q_shared)

    # intermediate rowmaxes
    m = torch.full((N,), -torch.inf, dtype=Q.dtype)
    m_shared = torch.empty((br, 1), dtype=Q.dtype)
    # intermediate normalization constants
    l = torch.full((N,), 0, dtype=Q.dtype)
    l_shared = torch.empty((br, 1), dtype=Q.dtype)

    for i in range(tc):
        # load k, v chunks
        # make sure we load in k as its transposed version
        K_shared[:, :] = K[:, i*bc:(i+1)*bc]
        V_shared[:, :] = V[i*bc:(i+1)*bc, :]

        for j in range(tr):
            # load in q, o, m, l
            Q_shared[:, :] = Q[j*br:(j+1)*br, :]
            # if i == 0: print(f'Q: {Q_shared}')
            O_shared[:, :] = O[j*br:(j+1)*br, :]
            m_shared[:, :] = m[j*br:(j+1)*br].unsqueeze(-1)
            l_shared[:, :] = l[j*br:(j+1)*br].unsqueeze(-1)
            
            S = Q_shared @ K_shared

            # get row-wise softmax statistics
            mt = S.max(dim=1).values.reshape(-1, 1)

            Pt = torch.exp(S - mt)
            lt = Pt.sum(dim=1).reshape(-1, 1)

            # compute new statistics
            m_new = torch.max(mt, m_shared)
            l_new = (torch.exp(m_shared - m_new) * l_shared) + (torch.exp(mt - m_new) * lt)


            # update chunk of output
            O_new = (l_shared * torch.exp(m_shared - m_new) * O_shared + torch.exp(mt - m_new) * Pt @ V_shared) / l_new
            O[j*br:(j+1)*br, :] = O_new
            
            m[j*br:(j+1)*br] = m_new.flatten()
            l[j*br:(j+1)*br] = l_new.flatten()

    return O






In [7]:
from math import sqrt
def dumb_attn(Q: torch.Tensor, K: torch.Tensor, V: torch.Tensor) -> torch.Tensor:
    """equivalent to F.scaled_dot_product_attention(Q, K, V, scale=1)"""
    d = Q.shape[1]
    return torch.softmax((Q)@ K.T, dim=1) @ V
    # return F.scaled_dot_product_attention(Q, K, V, scale=1)


# have to support up to like d=128
# Model dim of 4096 is divided ACROSS HEADS
# we're giving up and only supporting fp16 (like flashattn)
# Q = torch.rand((8196, 128), dtype=torch.float16, device='cuda') 
# K = torch.rand((8196, 128), dtype=torch.float16, device='cuda') 
# V = torch.rand((8196, 128), dtype=torch.float16, device='cuda') 

# K[-1] = torch.zeros((2,))
# V[-1] = torch.zeros((2,))


Q = torch.randint(1, 9, (8, 2), device='cuda').to(torch.float16)
K = torch.randint(1, 9, (8, 2), device='cuda').to(torch.float16)
V = torch.randint(1, 9, (8, 2), device='cuda').to(torch.float16)

# Q = torch.tensor([[0., 1.],
#          [2., 3.]])

# K = torch.tensor([[0., 1.5],
#          [2., 3.]])

# V = torch.tensor([[1., 0.],
#          [0., 1.]])
Q, K, V


(tensor([[7., 6.],
         [8., 5.],
         [4., 5.],
         [7., 2.],
         [4., 3.],
         [3., 1.],
         [4., 5.],
         [6., 2.]], device='cuda:0', dtype=torch.float16),
 tensor([[8., 3.],
         [4., 7.],
         [2., 8.],
         [2., 1.],
         [2., 1.],
         [1., 1.],
         [4., 7.],
         [8., 7.]], device='cuda:0', dtype=torch.float16),
 tensor([[2., 8.],
         [5., 7.],
         [3., 4.],
         [2., 7.],
         [2., 6.],
         [7., 4.],
         [2., 2.],
         [6., 4.]], device='cuda:0', dtype=torch.float16))

In [132]:
from torch.utils.cpp_extension import load
module = load(
    name='m',
    # sources=['cuda/main.cpp', 'cuda/flash_attention.cu'],
    sources=['rocm/flash_attention.hip',],
    extra_cflags=['--offload-arch="gfx1100"',],
    build_directory='build',
    verbose=True
)

/home/seb/Code/flash-attention/rocm/flash_attention.hip -> /home/seb/Code/flash-attention/rocm/flash_attention_hip.hip [ok]
Total number of unsupported CUDA function calls: 0


Total number of replaced kernel launches: 2


The input conditions for extension module m have changed. Bumping to version 9 and re-building as m_v9...
Successfully preprocessed all matching files.
Detected CUDA files, patching ldflags
Emitting ninja build file build/build.ninja...
Building extension module m_v9...
Allowing ninja to set a default number of workers... (overridable by setting the environment variable MAX_JOBS=N)


[1/2] /opt/rocm-6.1.3/bin/hipcc  -DWITH_HIP -DTORCH_EXTENSION_NAME=m_v9 -DTORCH_API_INCLUDE_EXTENSION_H -DPYBIND11_COMPILER_TYPE=\"_gcc\" -DPYBIND11_STDLIB=\"_libstdcpp\" -DPYBIND11_BUILD_ABI=\"_cxxabi1011\" -isystem /home/seb/Code/pyenvs/rocmenv/lib/python3.12/site-packages/torch/include -isystem /home/seb/Code/pyenvs/rocmenv/lib/python3.12/site-packages/torch/include/torch/csrc/api/include -isystem /home/seb/Code/pyenvs/rocmenv/lib/python3.12/site-packages/torch/include/TH -isystem /home/seb/Code/pyenvs/rocmenv/lib/python3.12/site-packages/torch/include/THC -isystem /home/seb/Code/pyenvs/rocmenv/lib/python3.12/site-packages/torch/include/THH -isystem /opt/rocm-6.1.3/include -isystem /home/seb/miniconda3/include/python3.12 -D_GLIBCXX_USE_CXX11_ABI=0 -fPIC -std=c++17 --offload-arch="gfx1100" -fPIC -D__HIP_PLATFORM_AMD__=1 -DUSE_ROCM=1 -DHIPBLAS_V2 -DCUDA_HAS_FP16=1 -D__HIP_NO_HALF_OPERATORS__=1 -D__HIP_NO_HALF_CONVERSIONS__=1 -fno-gpu-rdc -c /home/seb/Code/flash-attention/rocm/flash_at

Loading extension module m_v9...


In [28]:
# torch.isclose(dumb_attn(Q, K, V), flashattn(Q, K, V))
# dumb_attn(Q, K, V), flashattn(Q, K, V)

# F.scaled_dot_product_attention(Q, K, V, scale=1), flashattn(Q, K, V)
# module.flash_attn(Q, K, V)
torch.set_printoptions(profile='default', sci_mode=False)
r1 = dumb_attn(Q, K, V)
r2 = module.flash_attn(Q, K, V)
# (r1-r2).abs().mean()
r1, r2


# # r1, dumb_attn(Q.float(), K.float(), V.float()), r2
# # r1, r2[:50]
# dumb_attn(Q, K, V), module.flash_attn(Q, K, V)

  Shared Memory per Block: 65536
  Shared Memory Banks: 65536
  Warp Size: 32
  Max Threads per Block: 1024
  Constant Memory: 2147483647
  L2 Cache: 6291456

Launching kernel with smem size 7040


(tensor([[6.0000, 4.0000],
         [6.0000, 4.0000],
         [6.0000, 4.0000],
         [5.9961, 4.0000],
         [6.0000, 4.0000],
         [5.9258, 4.0703],
         [6.0000, 4.0000],
         [5.9961, 4.0000]], device='cuda:0', dtype=torch.float16),
 tensor([[6.0000, 4.0000],
         [6.0000, 4.0000],
         [6.0000, 4.0000],
         [5.9961, 4.0000],
         [6.0000, 4.0000],
         [5.9258, 4.0742],
         [6.0000, 4.0000],
         [5.9961, 4.0000]], device='cuda:0', dtype=torch.float16))

In [131]:

def test_flashattn():
    diff = lambda r1, r2: (r1-r2).abs().mean()
    dims = ((67, 128), (48, 128), (32, 128), (128, 128))
    # dims = ((48, 32), (64, 256), (1024, 256), (4096, 1024))
    for N, d in dims:
        Q = torch.rand((N, d), dtype=torch.float16, device='cuda') 
        K = torch.rand((N, d), dtype=torch.float16, device='cuda') 
        V = torch.rand((N, d), dtype=torch.float16, device='cuda') 

        # print(f'Difference on ({N}, {d}) tensors: {diff(dumb_attn(Q, K, V), F.scaled_dot_product_attention(Q, K, V, scale=1))}\n\n')
        res = module.flash_attn(Q, K, V)
        # print(torch.isnan(res).any())
        # print(res, res.shape)
        print(f'Difference on ({N}, {d}) tensors: {diff(res, F.scaled_dot_product_attention(Q, K, V, scale=1))}\n\n')

        # print(f'Difference on ({N}, {d}) tensors: {diff(module.flash_attn(Q, K, V), dumb_attn(Q, K, V))}\n\n')

torch.set_printoptions(profile='full', sci_mode=False)
test_flashattn()
torch.set_printoptions(profile='default', sci_mode=False)

  Shared Memory per Block: 65536
  Shared Memory Banks: 65536
  Warp Size: 32
  Max Threads per Block: 1024
  Constant Memory: 2147483647
  L2 Cache: 6291456

tc=2, tr=3
Launching kernel with smem size 44032
Difference on (67, 128) tensors: 0.0005469322204589844


  Shared Memory per Block: 65536
  Shared Memory Banks: 65536
  Warp Size: 32
  Max Threads per Block: 1024
  Constant Memory: 2147483647
  L2 Cache: 6291456

tc=1, tr=2
Launching kernel with smem size 44032
Difference on (48, 128) tensors: 0.0005464553833007812


  Shared Memory per Block: 65536
  Shared Memory Banks: 65536
  Warp Size: 32
  Max Threads per Block: 1024
  Constant Memory: 2147483647
  L2 Cache: 6291456

tc=1, tr=1
Launching kernel with smem size 44032
Difference on (32, 128) tensors: 0.0006723403930664062


  Shared Memory per Block: 65536
  Shared Memory Banks: 65536
  Warp Size: 32
  Max Threads per Block: 1024
  Constant Memory: 2147483647
  L2 Cache: 6291456

tc=3, tr=4
Launching kernel with smem size 440

In [ ]:
K = torch.tensor([0.3015, 0.8394, 0.2264, 0.8091, 0.9326, 0.7358, 0.9697, 0.0984, 0.0986, 0.4973, 0.5210, 0.3948, 0.3135, 0.9189, 0.1416, 0.9893, 0.8916, 0.1418, 0.0627, 0.1509, 0.2627, 0.0657, 0.5039, 0.5825, 0.6348, 0.1554, 0.8003, 0.8770, 0.8062, 0.1035, 0.7808, 0.0014, -torch.inf, -torch.inf, -torch.inf, -torch.inf, -torch.inf, -torch.inf, -torch.inf, -torch.inf, -torch.inf, -torch.inf, -torch.inf, -torch.inf, -torch.inf, -torch.inf, -torch.inf, -torch.inf, ]).to(torch.float16)
Q = 